# Rough Bright Spot Locator

In order to run this script, you will need to use the `histalign` environment located in your home directory (i.e., `miniconda3/envs/histalign`). To activate it, use `conda activate histalign` from the command line. Then navigate to the histalign programme directory on the Z drive. There, type `pip install -e .` (the dot is important).  
Then, type `pip install jupyter` to install the Jupyter set of tools to the environment (much simpler than trying to tie this environment back to JupyterLab).  
Then, make sure you've got all the packages listed in the first cell too. The first paragraph is built-ins so you will have them. The second paragraph is what you will need to install, the third was covered by `pip install -e .`.    
Finally, you should be able to use the environment to run the notebook. Don't forget to give the path to the alignment file, which should be absolute to avoid problems. 

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import peak_local_max

from histalign.backend.io import load_image
from histalign.backend.models import AlignmentSettings
from histalign.backend.registration import Registrator

## Load the alignment

In [ ]:
alignment_path = Path(
    "",  # the path to the alignment file (i.e., bunch of numbers and letters ending with .json)
)
with open(alignment_path) as handle:
    alignment_settings = AlignmentSettings(**json.load(handle))

## Load the image

Note that this is automatically determined from the alignment file, no need to enter it manually.

In [ ]:
image = load_image(alignment_settings.histology_path)

In [ ]:
plt.imshow(image);

## Generate the forward-registered image

In [ ]:
registered_image = Registrator(True, True).get_forwarded_image(
    image, alignment_settings
)

In [ ]:
plt.imshow(registered_image);

## Compute the coordinates

Play around with the `threshold_abs` value of `peak_local_max` if there are too many/not enough bright spots detected.

In [ ]:
coordinates = peak_local_max(registered_image, min_distance=100, threshold_abs=250)

figure, axes = plt.subplots(1, 1)

axes.imshow(registered_image)
axes.scatter(coordinates[:, 1], coordinates[:, 0], c="r")
axes.axis(False)

plt.show()

In [ ]:
coordinates

In [ ]:
ccf_coordinates = [
    alignment_settings.volume_settings.shape[0] // 2
    - alignment_settings.volume_settings.offset,
    *coordinates[0].tolist(),
]
ccf_coordinates

In [ ]:
# bregma = (540, 44, 570) @10um
bregma = np.round(np.array([540 / 2.5, 44 / 2.5, 227 / 2.5])).astype(np.uint16)
bregma = bregma.tolist()
bregma

In [ ]:
relative_to_bregma = np.array(ccf_coordinates) - np.array(bregma)
print(f"Position relative to bregma: {relative_to_bregma.tolist()}")